In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import warnings
from pyproj import Transformer

# Open the file correctly
cloud = xr.open_dataset(cp, engine="netcdf4", decode_times=False, mask_and_scale=True)

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import warnings
from pyproj import Transformer

cp = ""
var = ""


cloud = xr.open_dataset(cp, engine="netcdf4", decode_times=False, mask_and_scale=True)

lat = cloud["Latitude"].data
lon = cloud["Longitude"].data
dat = cloud[var].data

# Step 1: filter valid data points
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    qa = cloud["Quality_Assurance_5km"].astype("uint16")

cot_use = (qa[:, :, 1] >> 0) & 1
cot_mask = (cot_use == 1)

if cot_mask.sum().item() == 0:
    cloud.close()
    continue

mask = np.isfinite(lat) & np.isfinite(lon) & np.isfinite(dat) & cot_mask
lat = lat[mask]
lon = lon[mask]
dat = dat[mask]

# Step 2: filter data points inside JX province
xs, ys = transformer.transform(lon, lat)

pts = gpd.GeoSeries(gpd.points_from_xy(xs, ys), crs="EPSG:4527")
inside = pts.within(jx_proj).values
if inside.sum() == 0:
    cloud.close()
    continue

xs = xs[inside]
ys = ys[inside]
dat = dat[inside]

# Step 3: match with grids
 


In [1]:
# Author: Anora Wu
# Date: Jan 7th 2026
# Construct a panel data, with each day between 2020-2025 being the time variable and each grid being the identity. 
# Each identity has a geometry and a id. 
# Fill in the cloud seeding operation day and location into the time slots and the grid 

import geopandas as gpd
import pandas as pd
import numpy as np
import os
import warnings
from shapely.geometry import box
from pyproj import Transformer


# Change to your rawdata directory
os.chdir('/Users/anora/Team MG Dropbox/Wanru Wu/Cloudseeding_Anora')

### CONSTRUCT GRID ###

# Load and project JX polygon to EPSG:32650 
jx_poly = gpd.read_file('jiangxi/jiangxi_shape.shp').geometry.iloc[0]
jx_poly.bounds


(113.57276909724303, 24.488938802096126, 118.4821180557606, 30.079843918040275)

In [ ]:
#
# PPS Python ftps script
# Usage:  python script.py   (requires Python 3)
#

import os, sys, hashlib
from ftplib import FTP_TLS
import urllib
import urllib.request

downloadCount=0
skipCount=0
ftpConnection=None
forceHttps=False
userid='wanru@uchicago.edu'

def usage():
  print ('Usage:  python script.py [https]  (requires Python 3)')
  print ('Connects to the PPS ftps server and pulls files generated during order processing.')
  print ('Confirms download with a SHA checksum calculation.')
  print ('Will use https requests if connection to ftps server fails.')
  print ('Argument:  https - force https connection')

def hashfile(filename, blocksize=65536):
  hasher = hashlib.sha1()
  with open(filename, 'rb') as localfile:
    buf = localfile.read(blocksize)
    while len(buf) > 0:
      hasher.update(buf)
      buf = localfile.read(blocksize)
  return hasher.hexdigest()

def calcCksum(filename,cksum=None):
  if (cksum):
    sha = hashfile(filename)
    print (' cksum Pass' if cksum==sha else ' cksum FAIL')
  else:
    print ('.')

def get(filepath,cksum=None):
  if (ftpConnection):
    getFtpFile(filepath,cksum)
  else:
    getHttpsFile(filepath,cksum)

def getFtpFile(filepath,cksum=None):
  global ftpConnection,downloadCount,skipCount
  path,filename=os.path.split(filepath)
  ftpConnection.cwd(path)
  download=True
  ftpSize=ftpConnection.size (filename)
  # determine if file exists in local directory
  if (os.path.exists(filename)):
    # check size and cksum
    if (cksum):
      sha = hashfile(filename)
      download=(cksum != sha)
    else:
      filesize=os.path.getsize(filename)
      download=(ftpSize!=filesize)

  if (download):
    # if not exists or file checks do not match, get file.
    downloadCount+=1
    sys.stdout.write( str(downloadCount)+') Downloading '+filename+'   '+str(ftpSize)+' bytes')
    sys.stdout.flush()
    with open(filename, 'wb') as localfile:
      ftpConnection.retrbinary('RETR ' + filename, localfile.write, 1024)
    calcCksum(filename,cksum)
  else:
    print ('Already downloaded '+filename)
    skipCount+=1

def getHttpsFile(filepath,cksum=None):
  global downloadCount
  path,filename=os.path.split(filepath)
  with urllib.request.urlopen('https://arthurhouhttps.pps.eosdis.nasa.gov'+filepath) as response, open(filename, 'wb') as localfile:
    downloadCount+=1
    sys.stdout.write(str(downloadCount)+') Downloading '+filename)
    sys.stdout.flush()
    localfile.write(response.read())
    localfile.close()
    calcCksum(filename,cksum)

def getPpsFiles():
  global ftpConnection,forceHttps
  print ('Connecting to PPS')
  if forceHttps==False:
    try:
      ftpConnection = FTP_TLS('arthurhou.pps.eosdis.nasa.gov')
      print (ftpConnection.getwelcome())
      ftpConnection.login(userid,userid)
      ftpConnection.sendcmd('TYPE i')
      print ('Connected.  Getting files...')
    except Exception as e:
      print ('Failed to connect to the PPS ftps server due to ' + str(e))
      print ('Trying https')
      ftpConnection=None

  if (ftpConnection==None):
    password_mgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()
    password_mgr.add_password(None,'https://arthurhouhttps.pps.eosdis.nasa.gov',userid,userid)
    handler = urllib.request.HTTPBasicAuthHandler(password_mgr)
    opener = urllib.request.build_opener(handler)
    urllib.request.install_opener(opener)

  # The following is the list of PPS files to transfer:
  get('/gpmallversions/V07/2020/01/01/radar/2B.GPM.DPRGMI.CORRA2022.20200101-S095020-E112254.033194.V07A.HDF5','8d22ca9ef7262bcd784f34ea6d68b0a6d5b2d4c4')
  get('/gpmallversions/V07/2020/01/02/radar/2B.GPM.DPRGMI.CORRA2022.20200102-S194705-E211938.033216.V07A.HDF5','5f642f6616cea4dc6bfc7fa34576a88d9863c587')
  get('/gpmallversions/V07/2020/01/03/radar/2B.GPM.DPRGMI.CORRA2022.20200103-S094017-E111251.033225.V07A.HDF5','e9b7a0710f3c8190218328af62659773f7ec7760')
  get('/gpmallversions/V07/2020/01/03/radar/2B.GPM.DPRGMI.CORRA2022.20200103-S185546-E202819.033231.V07A.HDF5','0637421e16e362c4d74872dc27312a6041ae6d14')
  get('/gpmallversions/V07/2020/01/05/radar/2B.GPM.DPRGMI.CORRA2022.20200105-S184542-E201815.033262.V07A.HDF5','273b2d334eadb4c781c72c6c626226d4ef37678a')
  get('/gpmallversions/V07/2020/01/06/radar/2B.GPM.DPRGMI.CORRA2022.20200106-S083854-E101127.033271.V07A.HDF5','4ea10f1809c96febdec7b543e9fcc88cf6875c1b')
  get('/gpmallversions/V07/2020/01/06/radar/2B.GPM.DPRGMI.CORRA2022.20200106-S175422-E192655.033277.V07A.HDF5','f330c98b92c64ae9eb293d2308734e8230db5f10')
  get('/gpmallversions/V07/2020/01/08/radar/2B.GPM.DPRGMI.CORRA2022.20200108-S082848-E100121.033302.V07A.HDF5','da118ff8a4ee76a1a37f96f9d95ad55c7a36a29f')
  get('/gpmallversions/V07/2020/01/08/radar/2B.GPM.DPRGMI.CORRA2022.20200108-S174413-E191646.033308.V07A.HDF5','60681857a82c910b563ab6e40f443a15742acfe9')
  get('/gpmallversions/V07/2020/01/09/radar/2B.GPM.DPRGMI.CORRA2022.20200109-S073715-E090948.033317.V07A.HDF5','662f6c73f3da0c1aabd0d2309258a6ae52b4b936')
  get('/gpmallversions/V07/2020/01/11/radar/2B.GPM.DPRGMI.CORRA2022.20200111-S072635-E085907.033348.V07A.HDF5','4b35b18d519c404dfa0f8777fc3abd0a34bfc35a')
  get('/gpmallversions/V07/2020/01/11/radar/2B.GPM.DPRGMI.CORRA2022.20200111-S164156-E181428.033354.V07A.HDF5','6299dcfc8339e0d7c46218fbe127d77390a5bb7f')
  get('/gpmallversions/V07/2020/01/13/radar/2B.GPM.DPRGMI.CORRA2022.20200113-S163113-E180346.033385.V07A.HDF5','df78da2ac67aadf33e5125637a9da31dcb54cde4')
  get('/gpmallversions/V07/2020/01/14/radar/2B.GPM.DPRGMI.CORRA2022.20200114-S062415-E075647.033394.V07A.HDF5','a75254c0c7fa76eb8814867c254bfe81b22e8ea4')
  get('/gpmallversions/V07/2020/01/14/radar/2B.GPM.DPRGMI.CORRA2022.20200114-S153935-E171207.033400.V07A.HDF5','4ee5b4629723627b7d96e8f5a7b9732304399872')
  get('/gpmallversions/V07/2020/01/16/radar/2B.GPM.DPRGMI.CORRA2022.20200116-S061330-E074603.033425.V07A.HDF5','401a46f5be0e97c1b16018a1bf8277bb1df26286')
  get('/gpmallversions/V07/2020/01/16/radar/2B.GPM.DPRGMI.CORRA2022.20200116-S152851-E170123.033431.V07A.HDF5','afab3829113ee22f54a8106a60fe379cbce08181')
  get('/gpmallversions/V07/2020/01/17/radar/2B.GPM.DPRGMI.CORRA2022.20200117-S052151-E065423.033440.V07A.HDF5','0e7e863bd1b7112acf4634aa19bddac81e6f908d')
  get('/gpmallversions/V07/2020/01/19/radar/2B.GPM.DPRGMI.CORRA2022.20200119-S051105-E064338.033471.V07A.HDF5','42f21f60df38492d2f6f47c5fc8cffa827ea61f9')
  get('/gpmallversions/V07/2020/01/19/radar/2B.GPM.DPRGMI.CORRA2022.20200119-S142626-E155857.033477.V07A.HDF5','6cdca9ac6e6cf80125e3564e43c997f576385cc0')
  get('/gpmallversions/V07/2020/01/20/radar/2B.GPM.DPRGMI.CORRA2022.20200120-S041925-E055158.033486.V07A.HDF5','d7990d63a09eca4f19aeba27f39d2a61252bd885')
  get('/gpmallversions/V07/2020/01/21/radar/2B.GPM.DPRGMI.CORRA2022.20200121-S141538-E154810.033508.V07A.HDF5','298544cc10c461d6a454b5b4e3f4f01c25f74927')
  get('/gpmallversions/V07/2020/01/22/radar/2B.GPM.DPRGMI.CORRA2022.20200122-S040838-E054110.033517.V07A.HDF5','2df390ef68ac783afbf0c566b6188639bfd8b53c')
  get('/gpmallversions/V07/2020/01/22/radar/2B.GPM.DPRGMI.CORRA2022.20200122-S132358-E145630.033523.V07A.HDF5','44d26bc953f3e87c9c5d48e56f7a59431f864cbe')
  get('/gpmallversions/V07/2020/01/24/radar/2B.GPM.DPRGMI.CORRA2022.20200124-S131309-E144541.033554.V07A.HDF5','08c5e1417902e38e4a3f2d578ebd0ea9746d92db')
  get('/gpmallversions/V07/2020/01/25/radar/2B.GPM.DPRGMI.CORRA2022.20200125-S030608-E043840.033563.V07A.HDF5','2431270b86ae74e1cce6bee5b7a829b9de3550c3')
  get('/gpmallversions/V07/2020/01/25/radar/2B.GPM.DPRGMI.CORRA2022.20200125-S122128-E135400.033569.V07A.HDF5','f607d1792a78cf1a149e646207810511c2cb6d75')
  get('/gpmallversions/V07/2020/01/27/radar/2B.GPM.DPRGMI.CORRA2022.20200127-S025518-E042750.033594.V07A.HDF5','ae0e9f8088136e92cfa3be988e7d034d4c621f27')
  get('/gpmallversions/V07/2020/01/27/radar/2B.GPM.DPRGMI.CORRA2022.20200127-S121037-E134310.033600.V07A.HDF5','0c873112819111a3b5f30f6c2b1aa45cb7c5ca15')
  get('/gpmallversions/V07/2020/01/28/radar/2B.GPM.DPRGMI.CORRA2022.20200128-S020336-E033609.033609.V07A.HDF5','7cf6b4c4584efae9833f498f67dbd1b5d776ee1d')
  get('/gpmallversions/V07/2020/01/29/radar/2B.GPM.DPRGMI.CORRA2022.20200129-S115947-E133219.033631.V07A.HDF5','2927ffe6dd5b8e674d1aa85a6e40d418a42e3630')
  get('/gpmallversions/V07/2020/01/30/radar/2B.GPM.DPRGMI.CORRA2022.20200130-S015245-E032517.033640.V07A.HDF5','3387b18dda43cca6cfed430c0f62ed7991b0b7bf')
  get('/gpmallversions/V07/2020/01/30/radar/2B.GPM.DPRGMI.CORRA2022.20200130-S110804-E124036.033646.V07A.HDF5','95391ca4c4756a9ba8bc4b37ad851e81692ff2c5')

  # Transfer complete; close connection

  if ftpConnection:
    ftpConnection.quit()
  print ('Number of files downloaded: '+str(downloadCount))
  if (skipCount>0):
    print ('Number of files already downloaded: '+str(skipCount))

  sys.exit(0)


if __name__ == '__main__':
  if sys.version_info[0]<3:
    raise Exception('Must use Python 3')
  if (len(sys.argv)>1):
    if (sys.argv[1].find('http')!=-1):
      forceHttps=True
    else:
      usage()
      sys.exit(1)
  getPpsFiles()